In [7]:
import numpy as np
from typing import Tuple, List
import random

*With a single-segment of memory representing the array, the one-dimensional index into computer memory can always be computed from the N-dimensional index*

Let $n_i$ be the value of the ith index into an array whose shape is represented by the $N$ integers $d_i (i = 0\ .\ .\ . N − 1)$. Then, the one-dimensional index into a C-style contiguous array is
$$n^C = \sum_{i=0}^{N-1}n_i\prod^{N-1}_{j=i+1}d_j$$
while the one-dimensional index into a Fortran-style array is
$$n^F = \sum_{i=0}^{N-1}n_i\prod_{j=0}^{N-1}d_j$$
In these formulas we are assuming that
$$\prod_{j=k}^{m}d_j=d_k d_{k+1}...d_{m-1} d_m$$

Let’s see how they expand out for determining the one-dimensional index corresponding to the element (1, 3, 2) of a 4 × 5 × 6 array. If the array is stored as Fortran contiguous, then
$$n_F = n_0 \cdot (1) + n_1 \cdot (4) + n_2 \cdot (4 \cdot 5)$$
$$ = 1 + 3 \cdot 4 + 3 \cdot 20 = 53$$
On the other hand, if the array is stored as $C$ contiguous, then
$$n^{C}=n_0 \cdot (5 \cdot 6) + n_1 \cdot (6) + n_2 \cdot (1)$$
$$ = 1 \cdot 30 + 3 \cdot 6 + 2 \cdot 1 = 50$$

array is (4 x 5 x 6)
indices = (1, 3, 2)
C-style linear index = 1 * (5*6) + 3 * (6) + 2 * (1) = 50

In [8]:
indices = [...]
shape = [...]

def accumulate(arr): return (el := arr[0]) * accumulate(arr[1:]) if arr else 1

sum_ = 0
for i in range(len(indices)):
    if i<len(indices)-1: # if were not on the last element
        sum_ += indices[i] * accumulate(shape[i+1:]) # all remaining shapes
    else: # last element
        sum_ += indices[i]

linear_index_cstyle = sum([(indices[i] * accumulate(shape[i+1:]) if i < len(indices) - 1 else indices[i]) for i in range(len(indices))])

class NDArray:
    def __init__(self, shape: Tuple[int, ...], data: List[float]):
        self.size = accumulate(shape)
        if len(data) != self.size: raise Exception("Data size does not match shape")
        self.shape = shape
        self.data = data

    def _ndim_to_cstyle(self, indices: Tuple[int, ...]):
        return sum([(indices[i] * accumulate(shape[i+1:]) if i < len(indices) - 1 else indices[i]) for i in range(len(indices))])

    def __getitem__(self, indices: Tuple[int, ...]):
        if len(indices) != len(self.shape): raise Exception("Indices do not match shape")
        linear_index_cstyle = self._ndim_to_cstyle(indices)
        assert linear_index_cstyle < self.size, "Index out of bounds - something went wrong"
        return self.data[linear_index_cstyle]
    
    def __setitem__(self, indices: Tuple[int, ...], value: float):
        if len(indices) != len(self.shape): raise Exception("Indices do not match shape")
        linear_index_cstyle = self._ndim_to_cstyle(indices)
        assert linear_index_cstyle < self.size, "Index out of bounds - something went wrong"
        self.data[linear_index_cstyle] = value

    def __repr__(self):
        return f"NDArray(shape={self.shape}, data={self.data})"

TypeError: unsupported operand type(s) for +=: 'int' and 'ellipsis'

The formulas for the one-dimensional index of the N-dimensional arrays reveal what results in an important generalization for memory layout. Notice that each
formula can be written as

$$n^X = \sum^{N-1}_{i=1}n_i s^x_i$$

where $s^x_i$ gives the *stride* for dimension *i*. Thus, for C and Fortran arrays respectively we have:
$$s^C_i = \prod^{N-1}_{j=i+1}d_j=d_{i+1}d_{i+2}\cdots d_{N-1},$$
$$s^F_i = \prod^{i-1}_{j=0}d_j=d_0d_1\cdots d_{i-1}$$

The stride is how many **elements** in the underlying one-dimensional layout of the array one must jump in order to get to the next array element of a specific dimension in the N-dimensional layout. Thus, in a C-style 4x5x6 array one must jump over 30 elements to increment the first index by one, so 30 is the stride for the first dimension $s^C_0 = 30$. If, for each array, we define a strides Tuple with $N$ integers, the we have pre-computed and stored the important piece of how to map the N-dimensional index to the one-dimensional one used by the computer.

an index on an array always produces an ndarray object (but with different strides) - this is what a *view* is

In [47]:
def accumulater(arr): return (el := arr[0]) * accumulate(arr[1:]) if arr else 1
def flatten(iterable: List):
    if not isinstance(iterable, list): return [iterable]
    return [el for sublist in iterable for el in flatten(sublist)]

class Ndarray:
    def __init__(self, shape: Tuple[int, ...], data: List[float]):
        self.size = self.numel = accumulate(shape) # the total number of elements in the array
        self.shape: Tuple = shape # the shape of the array
        self.data: List = data # the buffer will do here
        self.stride_size: int = 8 #? if this = 1, strides is the number of elements to skip, if this = 8, strides is the number of bytes to skip
        self.strides: Tuple = self._calculate_strides()
        self.itemsize: int = 4 # itemsize is the number of bytes per element (dtype). thus 4*8 = 32 bits (float32)
        self.bytes = self.size * self.itemsize # total number of bytes in the array
        self.iscontiguous = True # if the array is contiguous in memory

        # validate
        if len(data) != self.size: raise Exception("Data size does not match shape")

    # strides
    def _calculate_strides(self):
        strides = [0 for _ in self.shape] # make an empty strides tuple
        for i in range(len(self.shape)): # for each dim
            stride = self.stride_size # start with the stride size
            for j in range(i+1, len(self.shape)): # accumulate the rest of the dims up to the N-1 dim
                stride *= self.shape[j]
            strides[i] = stride # set the stride in the Tuple
        return tuple(strides) # return the tuple
    
    def __getitem__(self, indices: Tuple[int,...]):
        def _check(self, indices): 
            if len(indices) != len(self.shape): raise Exception("Indices do not match shape")
            for i in range(len(indices)):
                if indices[i] >= self.shape[i]: raise Exception("Index out of bounds")
                if indices[i] < 0: raise Exception("Index out of bounds")
    
        _check(self, indices)

        linear_index = 0
        for i in range(len(indices)):
            linear_index += indices[i] * self.strides[i]
            
        return self.data[linear_index]

    @staticmethod
    def array(data: List):
        return NDArray((len(data),), data)
    
    def __repr__(self):
        return f"Ndarray(shape={self.shape}, strides={self.strides}, data={self.data})"
    

def test_ndarray():
    shape = (3, 4, 5)
    data = [random.randint(0, 10) for _ in range(accumulate(shape))]

    arr = Ndarray(shape, data)
    np_arr = np.array(data).reshape(shape)

    assert arr.data == flatten(np_arr.data.tolist()), "data mismatch"
    assert arr.shape == np_arr.shape, "shape mismatch"
    assert arr.strides == np_arr.strides, "strides mismatch"

test_ndarray()

In [29]:
shape = (3, 4, 5)
data = [random.randint(0, 10) for _ in range(accumulate(shape))]

arr = Ndarray(shape, data)
np_arr = np.array(data).reshape(shape)

In [38]:
np_arr.data.tolist().flatten()

AttributeError: 'list' object has no attribute 'flatten'